<h1>Submarine cable data scraping</h1>
This script fetches and analyzes data about submarine cables that were built and became ready for service between the years 2010 and 2024. It performs the following tasks:

1. Retrieves data for each year from an API.
2. Extracts the cable name and the year they became operational.
3. Saves the consolidated data to a CSV file.
4. Calculates the number of cables finalized per year and saves this summary to another CSV file.

In [1]:
import requests

# I start by scraping the URL for the cables built in 2010 to take a look at the dataset
url = "https://www.submarinecablemap.com/api/v3/ready-for-service/2010.json"

response = requests.get(url)

data = response.json()

data

{'id': '2010',
 'name': '2010',
 'landing_points': ['chindini-comoros',
  'mutsamudu-comoros',
  'fomboni-moheli-comoros',
  'haramous-djibouti',
  'mombasa-kenya',
  'dar-es-salaam-tanzania',
  'maputo-mozambique',
  'toliara-madagascar',
  'mtunzini-south-africa',
  'port-sudan-sudan',
  'moroni-comoros',
  'mogadishu-somalia',
  'lagos-nigeria',
  'bude-united-kingdom',
  'accra-ghana',
  'majuro-marshall-islands',
  'kwajalein-marshall-islands',
  'pohnpei-micronesia',
  'piti-guam',
  'kawaihae-hi-united-states',
  'papenoo-french-polynesia',
  'vaitape-french-polynesia',
  'uturoa-french-polynesia',
  'moorea-french-polynesia',
  'huahine-french-polynesia',
  'mumbai-india',
  'alexandria-egypt',
  'suez-egypt',
  'karachi-pakistan',
  'fujairah-united-arab-emirates',
  'catania-italy',
  'tripoli-lebanon',
  'marseille-france',
  'jeddah-saudi-arabia',
  'pontianak-indonesia',
  'banjarmasin-indonesia',
  'ketapang-indonesia',
  'sangata-indonesia',
  'beculuk-indonesia',
  'mat

In [2]:
import pandas as pd

df = pd.json_normalize(data['cables'])

In [3]:
# I calculate the length of the df to see if it matches the number of cables I've manually counted listed in the URL.
len(df)

16

In [4]:
# I take a look at the cables
df.head(16)

,id,name,rfs_year,is_planned
0,comoros-domestic-cable-system,Comoros Domestic Cable System,2010,False
1,eastern-africa-submarine-system-eassy,Eastern Africa Submarine System (EASSy),2010,False
2,glo-1,Glo-1,2010,False
3,hantru1-cable-system,HANTRU1 Cable System,2010,False
4,honotua,Honotua,2010,False
5,imewe,IMEWE,2010,False
6,jaka2ladema,JaKa2LaDeMa,2010,False
7,konstanz-meersburg,Konstanz-Meersburg,2010,False
8,mainone,MainOne,2010,False
9,mishima-village,Mishima Village,2010,False


In [5]:
df['name'],['rfs_year']

(0                       Comoros Domestic Cable System
 1             Eastern Africa Submarine System (EASSy)
 2                                               Glo-1
 3                                HANTRU1 Cable System
 4                                             Honotua
 5                                               IMEWE
 6                                         JaKa2LaDeMa
 7                                  Konstanz-Meersburg
 8                                             MainOne
 9                                     Mishima Village
 10                                            PGASCOM
 11        San Andres Isla Tolu Submarine Cable (SAIT)
 12    Suriname-Guyana Submarine Cable System (SG-SCS)
 13                        Tobrok-Emasaed Cable System
 14                                        Tverrlinken
 15                                  Unity/EAC-Pacific
 Name: name, dtype: object,
 ['rfs_year'])

In [6]:
# Now that I know that the data looks like, I can write code that allows me to loop through the URLs from 2010 to 2024 and fetch the data so I can download it.
years = range(2010, 2025)
base_url = "https://www.submarinecablemap.com/api/v3/ready-for-service/{}.json"

In [7]:
# Creating an empty list to store the data
all_cables = []

In [8]:
# Loop through the years
# Loop through the years and fetch data
for year in years:
    url = base_url.format(year)
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        # Normalize the JSON data to a DataFrame
        df = pd.json_normalize(data['cables'])
        # Select only the desired columns
        df = df[['name', 'rfs_year']]
        df = df.rename(columns={'rfs_year': 'year'})
        all_cables.append(df)
    else:
        print(f"Failed to fetch data for year {year}")

In [9]:
all_cables

[                                               name  year
 0                     Comoros Domestic Cable System  2010
 1           Eastern Africa Submarine System (EASSy)  2010
 2                                             Glo-1  2010
 3                              HANTRU1 Cable System  2010
 4                                           Honotua  2010
 5                                             IMEWE  2010
 6                                       JaKa2LaDeMa  2010
 7                                Konstanz-Meersburg  2010
 8                                           MainOne  2010
 9                                   Mishima Village  2010
 10                                          PGASCOM  2010
 11      San Andres Isla Tolu Submarine Cable (SAIT)  2010
 12  Suriname-Guyana Submarine Cable System (SG-SCS)  2010
 13                      Tobrok-Emasaed Cable System  2010
 14                                      Tverrlinken  2010
 15                                Unity/EAC-Pacific  20

In [10]:
# Check whether all years are in the df
len(all_cables)

15

In [11]:
# Put together all the data into a single df
final_allcables = pd.concat(all_cables, ignore_index=True)

In [12]:
# Save the df to an csv file
output_file = "submarine_cables_2010_2024.csv"
final_allcables.to_csv(output_file, index=False)

In [14]:
# The list of cable names and year when they were built is ultimately not what I want to visualize. I will now write code that will count the number of cables built per year and createas a new df I can download as a separate csv file.
cables_per_year = final_allcables['year'].value_counts().reset_index()
cables_per_year.columns = ['year', 'cable_count']
cables_per_year = cables_per_year.sort_values('year')

In [15]:
count_output_file = "cables_per_year_2010_2024.csv"
cables_per_year.to_csv(count_output_file, index=False)